In [6]:
# YouTube 댓글 다운로드를 위한 라이브러리 설치
!pip install youtube-comment-downloader

# 언어 감지를 위한 라이브러리 설치 (langid: 텍스트의 언어를 감지해주는 라이브러리)
!pip install langid

# Streamlit 설치 (Streamlit은 간단한 웹 앱을 빠르게 만들 수 있게 해주는 라이브러리)
!pip install streamlit

# ngrok을 통해 로컬 서버를 외부에서도 접속 가능하게 해주는 라이브러리 설치 (streamlit 웹앱을 배포할 때 사용 가능)
!pip install pyngrok

# 언어 코드 관련 처리를 위한 라이브러리 설치 (langcodes: ISO 639 언어 코드 관리 및 변환)
!pip install langcodes

In [4]:
#지금 작성한 코드 블록을 파일로 저장해 주는 기능
%%writefile streamlit_app.py

#____________________________________________________________________________

import streamlit as st  # Streamlit: 웹앱을 쉽게 만들 수 있는 라이브러리
import os  # OS 명령어 실행용
import json  # JSON 데이터 처리
import pandas as pd  # 데이터프레임 처리
import langid  # 언어 감지 라이브러리
import langcodes  # 언어 코드 -> 언어 이름 매핑 라이브러리

#____________________________________________________________________________


# 디자인 CSS 추가: 웹앱의 배경, 폰트, 버튼 등 시각적 스타일 정의

st.markdown("""
<style>
/* 배경 및 글꼴 설정 */
body {
  margin: 0;
  padding: 0;
  font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
  background-color: #ffffff;
  text-align: center;
  color: #333;
}
/* 중앙 로고 및 텍스트 스타일 */
.center-box {
  display: flex;
  flex-direction: column;
  justify-content: center;
  align-items: center;
  margin-top: 50px;
}
.center-box img {
  width: 40px;
  margin-bottom: 20px;
}
.center-box .title {
  font-size: 48px;
  color: #e60000;
  font-weight: bold;
  margin: 10px 0;
}
.center-box .subtitle {
  font-size: 16px;
  color: #888;
  margin-bottom: 30px;
}
/* 입력란과 버튼 스타일 */
.input-style input {
  padding: 12px 20px;
  width: 300px;
  border-radius: 10px;
  border: 1px solid #ccc;
  font-size: 14px;
}
.button-style button {
  padding: 12px 20px;
  background-color: #e60000;
  color: white;
  border: none;
  border-radius: 10px;
  font-weight: bold;
  cursor: pointer;
  font-size: 14px;
}
.button-style button:hover {
  background-color: #cc0000;
}
</style>
""", unsafe_allow_html=True)

# 중앙 로고, 제목, 설명 표시
st.markdown("""
<div class="center-box">
  <img src="https://upload.wikimedia.org/wikipedia/commons/b/b8/YouTube_Logo_2017.svg" alt="YouTube logo" />
  <div class="title">YouniversAI</div>
  <div class="subtitle">Your smart assistant for multilingual YouTube comment insights.</div>
</div>
""", unsafe_allow_html=True)

#____________________________________________________________________________


# YouTube 댓글을 다운로드하여 DataFrame으로 반환하는 함수
def get_comments(url):
    json_file = 'YoutubeComments.json'
    os.system(f'youtube-comment-downloader --url "{url}" --output {json_file}')  # 외부 명령어로 댓글 다운로드 (youtube-comment-downloader 사용)

    # JSON 파일 읽기
    with open(json_file, 'r', encoding='utf-8') as f:
        content = f.read()
        try:
            json_data = json.loads(content)  # 표준 JSON 파싱
        except json.JSONDecodeError:
            # JSON 라인 단위 파싱 (비표준 형식 처리)
            json_data = []
            for line in content.splitlines():
                line = line.strip()
                if not line:
                    continue
                try:
                    json_data.append(json.loads(line))
                except json.JSONDecodeError:
                    continue

    df = pd.DataFrame(json_data)  # pandas DataFrame 생성
    os.remove(json_file)  # 임시 파일 삭제
    return df

#____________________________________________________________________________


# 언어 감지 및 언어 이름 변환 함수
def classify_language(df):
    df['lang'] = df['text'].apply(lambda x: langid.classify(x)[0])  # langid로 언어코드 감지
    df['언어'] = df['lang'].apply(lambda code: langcodes.Language.get(code).display_name() if code else code)  # 언어 코드 -> 언어명 변환
    return df

#____________________________________________________________________________

# Streamlit 웹앱의 메인 함수
def main():
    # URL 입력란과 버튼
    url = st.text_input("Paste YouTube video URL here")
    if st.button("Go"):
        st.write(f"📺 You entered: {url}")

        with st.spinner("🔎댓글을 분류중입니다... 잠시만 기다려주세요"):
            df = get_comments(url)  # 댓글 가져오기
            if df.empty or 'text' not in df.columns:
                st.warning("No comments found or 'text' field missing.")  # 오류 처리
                return
            df = classify_language(df)  # 언어 분류
            st.session_state.df = df  # 세션 상태에 저장 (다른 인터랙션에서도 유지)

    # 댓글 데이터가 있는 경우 출력
    if 'df' in st.session_state:
        df = st.session_state.df
        st.success("댓글 수집 및 언어 분류 완료!")
        st.write("총 댓글 수:", len(df))

        # 언어 선택 멀티셀렉트
        languages = df['lang'].unique().tolist()  # 언어 코드 리스트
        language_options = [langcodes.Language.get(code).display_name() for code in languages]

        selected_langs = st.multiselect("언어를 선택하세요", options=language_options, default=language_options)

        # 선택된 언어에 해당하는 코드 필터링
        selected_lang_codes = [code for code in languages if langcodes.Language.get(code).display_name() in selected_langs]
        filtered_df = df[df['lang'].isin(selected_lang_codes)]

        # 선택된 언어의 댓글 출력
        if not filtered_df.empty:
            st.write(filtered_df[['언어', 'text']])
        else:
            st.warning("선택한 언어에 해당하는 댓글이 없습니다.")

#____________________________________________________________________________

# 프로그램 시작점
if __name__ == "__main__":
    main()


Overwriting streamlit_app.py


In [ ]:
from pyngrok import ngrok  # pyngrok 라이브러리를 임포트하여 ngrok과 연동


# ngrok 토큰 등록 (여기에 본인 ngrok 계정의 인증 토큰을 입력해야 함)
ngrok.set_auth_token("")



# 연결할 포트를 지정하고, ngrok으로 해당 포트를 외부에서 접속 가능하게 연결
port = 8501  # Streamlit의 기본 포트가 8501
public_url = ngrok.connect(port)  # 포트 8501을 외부 접속용으로 공개
print(f"Public URL: {public_url}")  # 생성된 public URL을 출력



# Streamlit 앱 실행 (streamlit_app.py 파일을 실행하고, 지정된 포트로 서버를 시작)
# &는 백그라운드 실행 (Colab 같은 환경에서는 필요)
!streamlit run streamlit_app.py --server.port {port} &
